In [4]:

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler, label_binarize
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, roc_curve, auc
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from docx import Document
from docx.shared import Inches, Pt
from datetime import datetime

# =========================
# 1. Load and clean dataset
# =========================
dataset_path = "/content/vgsalesGlobale.csv"   # make sure this file is in the same folder
plots_dir = "plots"
os.makedirs(plots_dir, exist_ok=True)

df = pd.read_csv(dataset_path)
df['Year'] = pd.to_numeric(df.get('Year'), errors='coerce')
df = df.dropna(subset=['Genre'])
df['Publisher'] = df['Publisher'].fillna('Unknown')
df['Year'] = df['Year'].fillna(int(np.nanmedian(df['Year'])))
for c in [c for c in df.columns if c.endswith('_Sales')]:
    df[c] = df[c].fillna(0)
df = df.drop_duplicates()
print("Data shape after cleaning:", df.shape)

# =========================
# 2. Exploratory Data Analysis
# =========================
plt.figure()
df['Global_Sales'].plot(kind='hist', bins=40, alpha=0.8)
plt.title('Distribution of Global Sales (Millions)')
plt.xlabel('Global_Sales')
plt.ylabel('Frequency')
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, "global_sales_hist.png"))
plt.close()

top_genre = df.groupby('Genre')['Global_Sales'].sum().sort_values(ascending=False).head(10).reset_index()
plt.figure()
plt.bar(top_genre['Genre'], top_genre['Global_Sales'])
plt.title('Top 10 Genres by Global Sales')
plt.xlabel('Genre'); plt.ylabel('Total Global Sales (Millions)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, "top_genres_global_sales.png"))
plt.close()

# Interactive Plotly chart
top_pub = df.groupby('Publisher')['Global_Sales'].sum().sort_values(ascending=False).head(10).reset_index()
fig = px.bar(top_pub, x='Publisher', y='Global_Sales', title='Top 10 Publishers by Global Sales (Interactive)')
fig.write_html("interactive_top_publishers.html")

# =========================
# 3. Feature Engineering
# =========================
y = df['Genre'].values
numeric_features = ['Year'] + [c for c in df.columns if c.endswith('_Sales') and c!='Global_Sales']
categorical_features = ['Platform']

# Limit publisher categories
N = 20
top_publishers = df['Publisher'].value_counts().head(N).index
df['Publisher_limited'] = np.where(df['Publisher'].isin(top_publishers), df['Publisher'], 'Other')
categorical_features.append('Publisher_limited')

X = df[numeric_features + categorical_features].copy()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

numeric_transformer = Pipeline([('scaler', StandardScaler())])
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocess = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# =========================
# 4. Baseline Models
# =========================
models = {
    "KNN": KNeighborsClassifier(),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "RandomForest": RandomForestClassifier(random_state=42)
}

baseline_results = {}
for name, clf in models.items():
    pipe = Pipeline([('preprocess', preprocess), ('model', clf)])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    baseline_results[name] = {
        "accuracy": acc,
        "report": classification_report(y_test, y_pred),
        "confusion_matrix": confusion_matrix(y_test, y_pred)
    }
    print(f"{name} baseline accuracy: {acc:.3f}")

# =========================
# 5. Feature Importance (RF)
# =========================
rf_pipe = Pipeline([('preprocess', preprocess), ('model', RandomForestClassifier(random_state=42))])
rf_pipe.fit(X_train, y_train)
rf_model = rf_pipe.named_steps['model']
ohe = rf_pipe.named_steps['preprocess'].named_transformers_['cat']
feature_names = numeric_features + list(ohe.get_feature_names_out(categorical_features))
importances = rf_model.feature_importances_
fi_df = pd.DataFrame({"feature": feature_names, "importance": importances}).sort_values("importance", ascending=False)

plt.figure()
plt.barh(fi_df["feature"].head(15)[::-1], fi_df["importance"].head(15)[::-1])
plt.title("Top 15 Feature Importances (Random Forest)")
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, "rf_feature_importances_top15.png"))
plt.close()

# =========================
# 6. Hyperparameter Tuning
# =========================
tuned_results = {}

# KNN
knn_pipe = Pipeline([('preprocess', preprocess), ('model', KNeighborsClassifier())])
knn_dist = {"model__n_neighbors": range(3, 21, 2), "model__weights": ["uniform", "distance"]}
knn_search = RandomizedSearchCV(knn_pipe, knn_dist, n_iter=5, cv=3, scoring="accuracy", random_state=42, n_jobs=-1)
knn_search.fit(X_train, y_train)
tuned_results["KNN"] = knn_search

# Decision Tree
dt_pipe = Pipeline([('preprocess', preprocess), ('model', DecisionTreeClassifier(random_state=42))])
dt_dist = {"model__max_depth": [None, 10, 20, 30], "model__min_samples_split": [2, 5, 10]}
dt_search = RandomizedSearchCV(dt_pipe, dt_dist, n_iter=5, cv=3, scoring="accuracy", random_state=42, n_jobs=-1)
dt_search.fit(X_train, y_train)
tuned_results["DecisionTree"] = dt_search

# Random Forest
rf_pipe = Pipeline([('preprocess', preprocess), ('model', RandomForestClassifier(random_state=42))])
rf_dist = {"model__n_estimators": [100, 200, 300], "model__max_depth": [None, 10, 20], "model__min_samples_split": [2, 5, 10]}
rf_search = RandomizedSearchCV(rf_pipe, rf_dist, n_iter=5, cv=3, scoring="accuracy", random_state=42, n_jobs=-1)
rf_search.fit(X_train, y_train)
tuned_results["RandomForest"] = rf_search

for name, search in tuned_results.items():
    acc = accuracy_score(y_test, search.predict(X_test))
    print(f"{name} tuned accuracy: {acc:.3f}, best params: {search.best_params_}")

# =========================
# 7. ROC Curve (best tuned model)
# =========================
best_model_name = max(tuned_results, key=lambda k: accuracy_score(y_test, tuned_results[k].predict(X_test)))
best_pipe = tuned_results[best_model_name].best_estimator_

classes = np.unique(y_train)
y_test_bin = label_binarize(y_test, classes=classes)

if hasattr(best_pipe.named_steps['model'], "predict_proba"):
    y_score = best_pipe.predict_proba(X_test)
    fpr, tpr, _ = roc_curve(y_test_bin.ravel(), y_score.ravel())
    roc_auc = auc(fpr, tpr)
    plt.figure()
    plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.3f}")
    plt.plot([0,1],[0,1],'--')
    plt.title(f"ROC Curve (micro-average) - {best_model_name}")
    plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
    plt.legend()
    plt.savefig(os.path.join(plots_dir, f"roc_curve_{best_model_name}.png"))
    plt.close()

# =========================
# 8. Generate Report (DOCX)
# =========================
doc = Document()
doc.add_heading("ML Assignment Report: Video Game Genre Prediction", level=1)
doc.add_paragraph(f"Date: {datetime.now().strftime('%Y-%m-%d')}")
doc.add_paragraph(f"Dataset size: {df.shape}")

doc.add_heading("EDA", level=2)
doc.add_paragraph("Global sales distribution is skewed; a few games dominate sales.")
doc.add_paragraph("Genres differ in sales totals (see plots).")

doc.add_heading("Models", level=2)
for name, res in baseline_results.items():
    doc.add_paragraph(f"{name} baseline accuracy: {res['accuracy']:.3f}")
for name, search in tuned_results.items():
    acc = accuracy_score(y_test, search.predict(X_test))
    doc.add_paragraph(f"{name} tuned accuracy: {acc:.3f}")

doc.add_heading("Best Model", level=2)
doc.add_paragraph(f"Best tuned model: {best_model_name}")

doc.save("ML_Assignment_Report.docx")
print("Report saved: ML_Assignment_Report.docx")


Data shape after cleaning: (16598, 11)
KNN baseline accuracy: 0.296
DecisionTree baseline accuracy: 0.257
RandomForest baseline accuracy: 0.292
KNN tuned accuracy: 0.302, best params: {'model__weights': 'uniform', 'model__n_neighbors': 11}
DecisionTree tuned accuracy: 0.308, best params: {'model__min_samples_split': 10, 'model__max_depth': 10}
RandomForest tuned accuracy: 0.339, best params: {'model__n_estimators': 300, 'model__min_samples_split': 10, 'model__max_depth': None}
Report saved: ML_Assignment_Report.docx


In [2]:
%pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 4.4 MB/s eta 0:00:00
